In [1]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.2/571.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.8 MB/s eta 0:00:00


In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    cloud_id="154429e62277481486a87de00a5ac969:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGQ3ODZhN2Q1N2IyMTRjY2ViNWU5MWRmZWNhMDIxNTg3JGIxMzJiZGFjZmZlNTQ3MGJiZGY3MDIwY2MyNWRiNGRm",
    api_key="S0R5ZC1aTUJ4MW5lOXZRQlh6Y2M6MUpHUU1BWGRSMUtJVmh3dHpuX0hnZw=="
)
if es.ping():
    print("Connected to Elasticsearch")
else:
    raise ConnectionError("Failed to connect to Elasticsearch")

Connected to Elasticsearch


In [3]:
import h5py
import pandas as pd

# Đường dẫn file
csv_path = '/kaggle/input/encodefinetune/second_half.csv'
h5_path = '/kaggle/input/encodefinetune/second_half_dvt_embeddings.h5'

# Đọc dữ liệu CSV
corpus_data = pd.read_csv(csv_path)

# Đọc dữ liệu từ HDF5
with h5py.File(h5_path, 'r') as h5_file:
    embeddings = h5_file['embeddings'][:]  # Lấy embeddings từ HDF5

In [4]:
embeddings.shape

(131627, 1024)

In [ ]:
index_name = "jin-ai-finetune1"

# Tạo mapping với các trường dense và sparse embeddings
mapping = {
    "mappings": {
        "properties": {
            "cid": {"type": "keyword"},  # keyword để lưu dạng string không phân tích cú pháp
            "text": {"type": "text"},
            "dense_embeddings": {
                "type": "dense_vector",
                "dims": 1024
            }
        }
    }
}

# Tạo index
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Tạo index '{index_name}' thành công.")
else:
    print(f"Index '{index_name}' đã tồn tại.")

In [5]:
from tqdm import tqdm
index_name = "jin-ai-finetune1"
# Lặp qua từng dòng trong corpus_data
for i, row in tqdm(corpus_data.iterrows(), desc="Uploading documents", unit="document"):
    document = {
        "cid": str(row["cid"]),  # Lấy CID từ CSV
        "text": row["text"],  # Văn bản từ CSV
        "dense_embeddings": embeddings[i].tolist(),  # Embeddings tương ứng từ HDF5
    }

    # Tải tài liệu lên Elasticsearch
    es.index(index=index_name, body=document)

print("Dữ liệu đã được tải lên Elasticsearch.")

Uploading documents: 131627document [1:40:25, 21.84document/s]

Dữ liệu đã được tải lên Elasticsearch.


In [6]:
# Kiểm tra số lượng tài liệu trong Elasticsearch
index_name = "jin-ai-finetune1"

response = es.count(index=index_name)

# In ra số lượng tài liệu
print(f"Số lượng tài liệu trong index '{index_name}': {response['count']}")

Số lượng tài liệu trong index 'jin-ai-finetune1': 263254
